<a href="https://colab.research.google.com/github/201921027/graduate_project_git/blob/main/_gp_bart_model_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [ ]:
import torch
import numpy as np

## CPU
device = torch.device("cpu")

## GPU
# device = torch.device("cuda")

In [ ]:
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ui1nzz00
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ui1nzz00


In [ ]:
# 사용자 input

# 서버에서 본문 받아오기
# 서버에서 아마 csv나 json으로 받아올텐데, 받아올 때 특수문자 처리 필수 (특히 따옴표 '', "")
# '[제목] 본문' 형태로 전처리
# //////////////////////서버로부터 본문 받아와서 str로 저장하는 line////////////////////////
input_text= "[아르바이트]한 달에 오 만원 정도의 용돈을 받아서 절반은 교통카드 충전을, \
또 절반은 친구들과 노는 곳에 주로 썼었다. 이 패턴은 고등학교를 다닐 때에도 마찬가지였다. \
한 달에 십 만원 정도의 용돈을 받아서 절반은 교통카드 충전 또 절반은 친구들과 노는 곳에 썼었지만 \
확실히 고등학교 때에는 돈이 모자르게만 느껴졌었고 아르바이트를 통해 부족한 돈을 충당하기보다는 \
버스를 타는 것 대신 삼십여분거리를 걸어서 등하교를 하고 그 교통비를 용돈에 보태서 쓰는 방법을 택했었다. \
어릴 때부터 용돈 받는선 일에 겁도 많이 났었지만 일단 한 번 해보자. 해봐야 뒷일도 아는 거니까라는 마음으로 \
첫 출근을 해서 일 년간 꾸준히 빵집 아르바이트를 통해 처음 저축도 하고 부모님께 선물도 사 드리고 \
내가 입고싶었던 옷, 핸드폰도 구입하고 보람찬 일들을 많이 했었기 때문에 \
아직도 그 빵집 앞을 지나갈 때나 혹은 익숙한 빵들을 보고 있을 때면 괜히 그때의 일들이 생각나 마음이 붕뜨고 설렌 적도 많았던 것 같다. \
부모님께 의존하고 부모님이 해주시는 모든 것들을 당연하게만 느꼈었던 과거의 나에게 지금의 내가 한마디 해줄 수 있다면 \
네가 받고 네가 누리고 있는 모든 것들에 항상 감사한 마음을 가지라고 해주고 싶다. \
별거 아니네라고 치부하며 낭비하지 말고 작은 것도 소중히 여겨 아낄 줄 아는 마음을 가진 내가 되었으면 좋겠다."


In [ ]:
if len(input_text) + len(" TL;DR ") > 800:
  input_text = input_text[0:400] + input_text[len(input_text)-400:len(input_text)]

* BERT

In [ ]:
from kobert import get_pytorch_kobert_model

bertmodel, vocab  = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
import gluonnlp as nlp
from kobert import get_tokenizer

bert_tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(bert_tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
from torch import nn

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
bert_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# model load
bert_model.load_state_dict(torch.load('./checkpoint/bert_model.pt', map_location=device))

<All keys matched successfully>

In [ ]:
max_len = 512

In [ ]:
# 기쁨:0 분노:1 혐오:2 두려움:3 슬픔:4 unknown:5

def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = ["기쁨", "분노", "혐오", "두려움", "슬픔", "unknown"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("일기의 감정:", cate[idx])
    print("신뢰도:", "{:.2f}%".format(softmax(result,idx)))
    return cate[idx]

In [ ]:
emotion= testModel(bert_model, input_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


일기의 감정: 기쁨
신뢰도: 90.09%


* KoBART

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

In [ ]:
model = BartForConditionalGeneration.from_pretrained('./KoBART-summarization/logs/model_chp')

In [ ]:
if len(input_text) + len(" TL;DR ") > 800:
  input_text = input_text[0:400] + input_text[len(input_text)-400:len(input_text)]

In [ ]:
input_text= input_text + " TL;DR "
text= emotion + '| ' + input_text

text = text.replace('\n', ' ')
# print(text)

In [ ]:
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
output= tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [ ]:
print(output)

아르바이트를 통해 부족한 돈을 충당하기보다는 버스를 타는 것 대신 삼십여분거리를 걸어서 등하교를 하고 그 교통비를 용돈에 보태서 쓰는 방법을 택했군요. 분명 좋은 결과가 있을 거에요!
